<a href="https://colab.research.google.com/github/Mohitnaik21/Agentic-AI-API-Builder/blob/main/Agentic_AI_API_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai sentence-transformers faiss-cpu fastapi uvicorn nest-asyncio pyngrok transformers accelerate torch google-generativeai


In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
from google.colab import drive


In [3]:
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
import google.generativeai as genai

GOOGLE_API_KEY = "Add your key here"
genai.configure(api_key=GOOGLE_API_KEY)

# models = genai.list_models()
# for model in models:
#     print(model.name)



In [11]:
# Download Public API dataset from GitHub
url = "https://raw.githubusercontent.com/public-apis/public-apis/master/README.md"
response = requests.get(url)

# Extract API Names & Descriptions
api_data = []
for line in response.text.split("\n"):
    if "| [" in line and "](https://" in line:
        parts = line.split("|")
        if len(parts) > 2:
            api_name = parts[1].split("[")[1].split("]")[0]
            api_url = parts[1].split("(")[1].split(")")[0]
            api_desc = parts[2].strip()
            api_data.append({"API Name": api_name, "API Endpoint": api_url, "Description": api_desc})

# Save as CSV
df = pd.DataFrame(api_data)
df.to_csv("public_apis.csv", index=False)

print("✅ Public API dataset saved as 'public_apis.csv'")


✅ Public API dataset saved as 'public_apis.csv'


In [35]:
class APIRetrievalAgent:
    def __init__(self):
        # Load embedding model
        self.embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

        # Load API dataset
        file_path = "public_apis.csv"
        self.api_df = pd.read_csv(file_path)
        self.api_df["text"] = self.api_df["API Name"] + " - " + self.api_df["Description"]

        # Generate embeddings
        api_texts = self.api_df["text"].tolist()
        api_embeddings = self.embedding_model.encode(api_texts, convert_to_numpy=True)

        # Create FAISS index
        dimension = api_embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatL2(dimension)
        self.faiss_index.add(api_embeddings)

        # Store metadata
        self.api_metadata = self.api_df[["API Name", "API Endpoint", "Description"]]

    def retrieve_apis(self, project_request, top_k=5):
        """Retrieve relevant APIs based on user request."""
        query_embedding = self.embedding_model.encode([project_request], convert_to_numpy=True)
        distances, indices = self.faiss_index.search(query_embedding, top_k)

        results = []
        for idx in indices[0]:
            if idx >= 0:
                api_data = self.api_metadata.iloc[idx].to_dict()
                results.append(api_data)

        return results


In [38]:
class BackendCodeAgent:
    def generate_backend_code(self, api_list):
        """Generate FastAPI backend code integrating selected APIs."""

        if not GOOGLE_API_KEY:
            raise ValueError("Google API key is missing. Set GOOGLE_API_KEY before running.")

        prompt = """You are a backend developer. Generate a Python FastAPI application that interacts with the following APIs:

        Requirements:
        - Do not copy existing code from external sources.
        - Write original and well-structured FastAPI code.
        - Ensure API calls are properly handled.
        - Return responses as JSON.
        - Implement error handling.

        The APIs to integrate are:\n\n"""

        for api in api_list:
            prompt += f"- {api['API Name']} ({api['API Endpoint']}): {api['Description']}\n"

        # Use less restrictive model & handle empty response
        model = genai.GenerativeModel("gemini-1.5-flash-latest")  # Change model to flash for more flexibility
        response = model.generate_content(prompt)
        # Debug: Print full response to see what is happening
        print("🔹 Raw AI Response (Backend):", response)


        if not response.candidates or not response.candidates[0].content.parts:
            raise ValueError("⚠️ AI model returned an empty response. Try modifying the prompt or using a different model.")


        print("\n✅ Generated Backend Code:\n", response.text)

        return response.text


In [39]:
class FrontendCodeAgent:
    def generate_frontend_code(self, backend_urls):
        """Generate a React frontend that fetches data from multiple backend APIs."""

        if not GOOGLE_API_KEY:
            raise ValueError("Google API key is missing. Set GOOGLE_API_KEY before running.")

        prompt = """You are a frontend React developer. Write a React component that:

        - Fetches data from multiple FastAPI endpoints.
        - Uses React hooks (`useState`, `useEffect`).
        - Displays API responses in a table.
        - Handles errors and loading states.

        APIs to integrate:\n\n"""

        for api_name, backend_url in backend_urls.items():
            prompt += f"- {api_name}: Fetch data from {backend_url}\n"

        # Use less restrictive model & handle empty response
        model = genai.GenerativeModel("gemini-1.5-flash-latest")
        response = model.generate_content(prompt)
        # Debug: Print full response to see what is happening
        print("🔹 Raw AI Response (Frontend):", response)

        if not response.candidates or not response.candidates[0].content.parts:
            raise ValueError("⚠️ AI model returned an empty response. Try modifying the prompt or using a different model.")

        print("\n✅ Generated Frontend Code:\n", response.text)
        return response.text


In [28]:
class ProjectManagerAgent:
    def __init__(self):
        self.api_agent = APIRetrievalAgent()
        self.backend_agent = BackendCodeAgent()
        self.frontend_agent = FrontendCodeAgent()

    def build_full_stack_app(self, project_request):
        print("📌 Understanding the request...")
        relevant_apis = self.api_agent.retrieve_apis(project_request, top_k=5)

        if not relevant_apis:
            return "❌ No relevant APIs found!"

        print("\n✅ APIs Retrieved:")
        for api in relevant_apis:
            print(f"🔹 {api['API Name']} → {api['API Endpoint']}")

        print("\n🚀 Generating Backend Code...")
        backend_code = self.backend_agent.generate_backend_code(relevant_apis)
        print("\n✅ Backend Code Generated.", backend_code)

        backend_urls = {api["API Name"]: f"http://127.0.0.1:8000/{api['API Name'].replace(' ', '_').lower()}" for api in relevant_apis}

        print("\n🎨 Generating Frontend Code...")
        frontend_code = self.frontend_agent.generate_frontend_code(backend_urls)
        print("\n✅ Frontend Code Generated.", frontend_code)

        return backend_code, frontend_code


In [40]:
pm_agent = ProjectManagerAgent()
backend_code, frontend_code = pm_agent.build_full_stack_app(
    "Build a weather dashboard using public APIs."
)


📌 Understanding the request...

✅ APIs Retrieved:
🔹 weather-api → https://github.com/robertoduessmann/weather-api
🔹 Tomorrow → https://docs.tomorrow.io
🔹 WeatherAPI → https://www.weatherapi.com/
🔹 apilayer weatherstack → https://weatherstack.com/
🔹 Weatherstack → https://weatherstack.com/?utm_source=Github&utm_medium=Referral&utm_campaign=Public-apis-repo-Best-sellers

🚀 Generating Backend Code...
🔹 Raw AI Response (Backend): response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "This solution demonstrates the basic structure for interacting with multiple weather APIs.  Due to the limitations of providing API keys and the varying authentication methods of the listed APIs, I cannot fully implement requests to all of them.  This example shows how to structure your FastAPI application to handle multiple API calls, 